In [1]:
!pip install -q transformers datasets matplotlib
!!pip install -q accelerate torchinfo huggingface_hub

[]

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.8
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...
Unable to run: 
    curl https://packagecloud.io/install/repositories/github/git-lfs/config_file.list?os=Ubuntu&dist=jammy&source=script

Double check your curl installation and try again.


main: line 154: /etc/apt/sources.list.d/github_git-lfs.list: Permission denied
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter
import torch 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast
from tqdm.notebook import tqdm
from transformers import T5ForConditionalGeneration, T5Config, AdamW
from tqdm.auto import tqdm
from accelerate import Accelerator

In [4]:
file_id = '1e1Tz4VycXhimx3ZyrxgL72VBaC0JP6By'
destination = 'human_proteins.tsv'
hf_name = "t5-small"
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

In [5]:
file_id = '1e1Tz4VycXhimx3ZyrxgL72VBaC0JP6By'
destination = 'human_proteins.tsv'

def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")


download_file_from_google_drive(file_id, destination)

0it [00:00, ?it/s]



saved all the data to human_proteins.tsv. total size : 33.218 MB


In [6]:
download_file_from_google_drive("1FLrC9kK5-R_NwjmX_WTqT8YqchzS9zIN", "all_proteins.tsv")

0it [00:00, ?it/s]



saved all the data to all_proteins.tsv. total size : 294.963 MB


In [7]:
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")
dataframe = dataframe[dataframe["sequence"].str.len() <= 510]
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")
dataframe = dataframe[dataframe["sequence"].str.len() <= 510]
dataframe["protein_name"] = [v.split("(")[0] for v in dataframe["protein_name"]]
dataframe["sequence"] = [" ".join(v) for v in dataframe["sequence"]]

with open("Sequences.txt", "w") as f:
    f.write("\n".join(dataframe["sequence"].values))
    
dataframe

,sequence,protein_name,length
0,M R W Q E M G Y I F Y P R K L R,Mitochondrial-derived peptide MOTS-c,16
1,M P G W F K K A W Y G L A S L L S F S S F I L ...,Clarin-2,232
2,M K R R Q K R K H L E N E E S Q E T A E K G G ...,Protein FAM170A,330
3,M D D A D P E E R N Y D N M L K M L S D L N K ...,Synaptonemal complex central element protein 3,88
4,M A F S D L T S R T V H L Y D N W I K D A D P ...,Elongation of very long chain fatty acids prot...,281
...,...,...,...
89896,M E P V D P R L E P W K H P G S Q P K T P C T ...,Protein Tat,101
89897,M G G K W S K R S A P G W N A V R E R M R R T ...,Protein Nef,206
89898,M L L L G A L L L L L A L P S Y G Q D T M Q G ...,Adiponectin,239
89899,M G Q F I S F M Q E I P I F L Q E A L N I A L ...,Pre-glycoprotein polyprotein GP complex,483


In [8]:
dataframe2 = pd.read_csv("all_proteins.tsv", sep="\t")
dataframe2 = dataframe2[dataframe2["sequence"].str.len() <= 510]
dataframe2 = dataframe2[dataframe2["sequence"].str.len() <= 510]
dataframe2["protein_name"] = [v.split("(")[0] for v in dataframe2["protein_name"]]

with open("names.txt", "w") as f:
    f.write("\n".join([v.split("(")[0].strip() for v in dataframe2["protein_name"].values]))  

dataframe2

,Unnamed: 0,sequence,protein_name,length
0,0,MRWQEMGYIFYPRKLR,Mitochondrial-derived peptide MOTS-c,16
1,1,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,Clarin-2,232
2,2,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,Protein FAM170A,330
3,3,MDDADPEERNYDNMLKMLSDLNKDLEKLLEEMEKISVQATWMAYDM...,Synaptonemal complex central element protein 3,88
4,4,MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,Elongation of very long chain fatty acids prot...,281
...,...,...,...,...
619007,529106,MNSNAPAAVIVLAAGAGTRMKSKLPKVLHEIGGRSLLMHAITAARG...,Bifunctional protein GlmU [Includes: UDP-N-ace...,492
619008,529107,MSATGSDPSRRPVDLPDLSREAVPGEKVALAPGQLQLRPTRRGKAP...,Probable dual-specificity RNA methyltransferas...,428
619009,529108,MVLASHNAKKLRELQRILAPAVPGLEAEQIVSAAGLGLPDVVEDAV...,dITP/XTP pyrophosphatase,203
619010,529109,MLPVLTADALRTAEQAHWDEHPGDDLMGRAAAEVARHATEMLGDGP...,ADP-dependent,492


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mlewand/PROT5-small-v2", max_length=512,
                                                   padding=True)

d:\Python\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lewan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [41]:
ids = tokenizer.encode("MRWQEMGYIFYPRKLR")
print(ids, tokenizer.decode(ids))
ids = tokenizer.encode("Protein FAM170A")
print(ids, tokenizer.decode(ids))

[1, 7124, 47, 41, 3692, 31, 49, 6635, 40, 2194, 3556, 2] <s> MRWQEMGYIFYPRKLR</s>
[1, 249, 9701, 12, 25, 2] <s> Protein FAM170A</s>


In [10]:
ids

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lewan\AppData\Local\Temp\ipykernel_13244\1866636969.py:1 in <module>                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\lewan\\AppData\\Local\\Temp\\ipykernel_13244\\1866636969.py'                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ids' is not defined

In [11]:
_tokenized_X = []
_tokenized_Y = []

tokenized_X = []
tokenized_Y = []
sizes_X = []
sizes_Y = []

max_length_inp = 512
max_length_out = 64

for x in tqdm(dataframe["sequence"]):
    input_tokens = tokenizer.encode(x)
    tokenized_X.append(input_tokens)
    sizes_X.append(len(input_tokens))

for y in tqdm(dataframe["protein_name"]):
    out_tokens = tokenizer.encode(y)
    tokenized_Y.append(out_tokens)
    sizes_Y.append(len(out_tokens))
    
for x,y in tqdm(zip(_tokenized_X, _tokenized_Y)):
   if len(x) <= max_length_inp and len(y) <= max_length_out:
    tokenized_X.append(x)
    tokenized_Y.append(y)

  0%|          | 0/89901 [00:00<?, ?it/s]

  0%|          | 0/89901 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [12]:
train_val_split = 0.9
l = len(tokenized_X)
split_idx = int(train_val_split * l)

idxes = np.arange(l)
r = np.random.permutation(idxes)

train_idxs = r[:split_idx]
val_idxs = r[split_idx:]

train_X, train_Y = [], []
val_X, val_Y = [], []


for idx in train_idxs:
    train_X.append(tokenized_X[idx])
    train_Y.append(tokenized_Y[idx])
    
for idx in val_idxs:
    val_X.append(tokenized_X[idx])
    val_Y.append(tokenized_Y[idx])   
    
print(f"train sizes ;  X : {len(train_X)}, Y : {len(train_Y)}")
print(f"val sizes ;  X : {len(val_X)}, Y : {len(val_Y)}")


train sizes ;  X : 80910, Y : 80910
val sizes ;  X : 8991, Y : 8991


In [20]:
class CustomAADataset(Dataset):
  def __init__(self, X, Y, max_length=512, max_length2=32):
    self.max_length = max_length
    self.max_length2 = max_length2 
    self.X = X
    self.Y = Y
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    input_tokens = self.X[idx]
    output_tokens = self.Y[idx]

    # Pad input and output tokens
    input_tokens = input_tokens + [0] * (self.max_length - len(input_tokens))

    output_tokens = output_tokens + [0] * (self.max_length2 - len(output_tokens))

    return {"input_ids": torch.tensor(input_tokens, dtype=torch.long),
            "labels": torch.tensor(output_tokens, dtype=torch.long),
            }


In [13]:
# Load data
from datasets import Dataset as Dataset_hf

dataset = Dataset_hf.from_dict({"source": dataframe["sequence"].values, "target": dataframe["protein_name"].values})

In [21]:
train_dataset = CustomAADataset(train_X, train_Y, max_length_inp, max_length_out)
val_dataset = CustomAADataset(val_X, val_Y, max_length_inp, max_length_out)

train_dataset[0]

{'input_ids': tensor([  1, 118, 100, 100, 104,  85, 104, 104, 109, 104, 110, 110, 115, 101,
         101,  92,  92,  88,  92,  90,  92, 104,  92, 109, 116, 111, 100, 104,
         110, 101, 116, 104, 131,  92, 131, 109, 129,  85,  90, 109,  92, 108,
          85,  85,  89,  90, 100, 101,  90,  90,  92,  90,  87,  89,  92, 104,
         104,  85,  90, 100, 110, 118, 100, 104,  92, 116,  85, 101,  88,  85,
         100, 109, 101,  85, 108, 118,  85,  89, 108,  87, 118, 101,  87, 104,
         101,  99, 109,  90, 104,  92, 115,  85, 110, 108,  92, 131,  88,  89,
         100, 101, 116, 116,  92, 100, 100, 115, 100,  89,  89, 110,  92, 116,
          88, 109,  85,  85,  85, 131, 101, 108,  87,  92,  88, 100, 109, 118,
         109, 116,  89,  85,  85, 104,  89,  92, 100, 109,  90,  92,  89, 101,
         108, 108,  99, 100, 100, 109, 110, 104, 104,  89,  85, 100, 109, 131,
         131, 104,  87, 109, 100,  88, 101, 110,  87, 115, 131, 100, 104,  89,
          92, 110, 101,  89, 104, 101, 

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=None, padding=True, max_length=512)

model_name = "mlewand/PROT5-small-v2"
config = T5Config.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True)

In [16]:
class CustomAADataset(Dataset):
  def __init__(self, X, Y, max_length=512, max_length2=32):
    self.max_length = max_length
    self.max_length2 = max_length2 
    self.X = X
    self.Y = Y
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    input_tokens = self.X[idx]
    output_tokens = self.Y[idx]

    # Pad input and output tokens
    input_tokens = input_tokens + [0] * (self.max_length - len(input_tokens))

    output_tokens = output_tokens + [0] * (self.max_length2 - len(output_tokens))

    return {"input_ids": torch.tensor(input_tokens, dtype=torch.long),
            "labels": torch.tensor(output_tokens, dtype=torch.long),
            }


In [22]:
tokenizer.decode((val_dataset[5]["labels"]))

'<s> HRas proto-oncogene, GTPase</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [23]:
def decode(pred):
    mask = pred != 0
    txt = tokenizer.decode(pred)
    txt = txt.replace("<pad>", "")
    txt = txt.replace("<s>", "")
    txt = txt.replace("</s>", "")
    return txt.strip()
    
model = model.to("cpu")
val_dl = DataLoader(val_dataset, batch_size=1, shuffle=False)

exact_match = 0
i = 0
idx = 600



with tqdm(val_dl) as pb:
    for i,batch in enumerate(val_dl):
        
        #if i != idx:
        #  continue

        ret = model(**batch)
        pred = torch.argmax(ret.logits, axis=-1).reshape(-1)
    
        exact_match += torch.all(pred == batch['labels']).item()
        i += 1
        # print(decode(pred), decode(batch['labels']))
        pb.update(1)
        pb.set_description(f"{exact_match}/{i} ; Acc : {round( 100. * exact_match / i , 2)}")
        true_dec = decode(batch['labels'][0])
        pred_dec = decode(pred)

        if true_dec == pred_dec and true_dec != "Protein Nef":
          print(f"idx : {i} | {true_dec} | {pred_dec}")


print(f"{exact_match}/{i}")

  0%|          | 0/8991 [00:00<?, ?it/s]

idx : 5 | Protein Tat | Protein Tat
idx : 9 | Protein Tat | Protein Tat
idx : 10 | Prostaglandin reductase 1 | Prostaglandin reductase 1
idx : 19 | Prostaglandin reductase 1 | Prostaglandin reductase 1
idx : 22 | Phosphatidylinositol 3,4,5-trisphosphate 3-phosphatase and dual-specificity protein phosphatase PTEN | Phosphatidylinositol 3,4,5-trisphosphate 3-phosphatase and dual-specificity protein phosphatase PTEN
idx : 24 | RING-type E3 ubiquitin transferase | RING-type E3 ubiquitin transferase
idx : 26 | Protein Tat | Protein Tat
idx : 29 | Elongation of very long chain fatty acids protein 6 | Elongation of very long chain fatty acids protein 6
idx : 32 | Protein Tat | Protein Tat


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lewan\AppData\Local\Temp\ipykernel_13244\3340833753.py:24 in <module>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\lewan\\AppData\\Local\\Temp\\ipykernel_13244\\3340833753.py'                         │
│                                                                                                  │
│ d:\Python\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl                        │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ d:\Python\lib\site-packages\transformers\models\t5\modeling_t5.py:1679 in forward                │
│                                                                                                  │
│   1676 │   │   # Encode if needed (training, first prediction pass)                              │
│   1677 │   │   if encoder_outputs is None:                                                       │
│   1678 │   │   │   # Convert encoder inputs in embeddings if needed                              │
│ ❱ 1679 │   │   │   encoder_outputs = self.encoder(                                               │
│   1680 │   │   │   │   input_ids=input_ids,                                                      │
│   1681 │   │   │   │   attention_mask=attention_mask,                                            │
│   1682 │   │   │   │   inputs_embeds=inputs_embeds,                                              │
│                                                                                                  │
│ d:\Python\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl                        │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ d:\Python\lib\site-packages\transformers\models\t5\modeling_t5.py:1086 in forward                │
│                                                                                                  │
│   1083 │   │   │   │   │   None,  # past_key_value is always None with gradient checkpointing    │
│   1084 │   │   │   │   )                                   

In [42]:
i = 10
print(val_dataset[i]["labels"])
v = model.generate(val_dataset[i]["input_ids"].reshape(1, -1), max_new_tokens=20)
print(tokenizer.decode(val_dataset[i]["labels"]))
print(tokenizer.decode(v[0]))

tensor([   1, 6136, 2760,  532,   80,    2,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
<s> Citrate synthase, mitochondrial </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<pad><s>


In [19]:
idx = 155

def decode(pred):
    mask = pred != 0
    txt = tokenizer.decode(pred)
    txt = txt.replace("<pad>", "")
    txt = txt.replace("<s>", "")
    txt = txt.replace("</s>", "")
    return txt.strip()
    
model = model.to("cpu")
val_dl = DataLoader(val_dataset, batch_size=1, shuffle=False)

exact_match = 0
idx = 154

with tqdm(val_dl) as pb:
    for i,batch in enumerate(val_dl):
        
        if i != idx:
          continue

        ret = model(**batch)
        pred = torch.argmax(ret.logits, axis=-1).reshape(-1)
    
        exact_match += torch.all(pred == batch['labels']).item()
        # print(decode(pred), decode(batch['labels']))
        pb.update(1)
        pb.set_description(f"{exact_match}/{i} ; Acc : {round( 100. * exact_match / i , 2)}")
        true_dec = decode(batch['labels'][0])
        pred_dec = decode(pred)

        v = "".join(decode(batch['input_ids'][0]).split(" "))

        print(f"input : {v}")
        print(f"idx : {i} | {true_dec} | {pred_dec}")
        break


print(f"{exact_match}/{i}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lewan\AppData\Local\Temp\ipykernel_13244\1287072851.py:12 in <module>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\lewan\\AppData\\Local\\Temp\\ipykernel_13244\\1287072851.py'                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'val_dataset' is not defined

In [ ]:
tokenizer.push_to_hub("mlewand/PROT5-small-v2")
#model.push_to_hub("mlewand/PROT5-small-v2")


CommitInfo(commit_url='https://huggingface.co/mlewand/PROT5-small-v2/commit/b9b96bdfe83002baad22a882a4519aad615e70a4', commit_message='Upload tokenizer', commit_description='', oid='b9b96bdfe83002baad22a882a4519aad615e70a4', pr_url=None, pr_revision=None, pr_num=None)